In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from patsy import dmatrices
import dispersion_multi_wrapper_ipt as DISP

In [2]:
raw_count = pd.read_csv('maqsAllCounts.csv', index_col=0)
Norm_NM_000014_4 = pd.DataFrame({"JOA1": [0],"JOA2": [4.697854],"JOA3": [3.063885],"JOA4": [0],"JOA5": [0.937649],"JOB1": [0],"JOB2": [0],"JOB3": [0.620733],"JOB4": [0.937502],"JOB5": [1.656273]}, index=["NM_000014_4"])
Norm_NM_000014_4_T = np.transpose(Norm_NM_000014_4)
Norm_NM_000014_4_T.insert(0, "type", ['univ', 'univ', 'univ', 'univ', 'univ', 'brain', 'brain', 'brain', 'brain', 'brain'])
feckless, designmatrix = dmatrices( "NM_000014_4~ C(type)", Norm_NM_000014_4_T, return_type="matrix")

In [3]:
MAQC_disp = DISP.Dispersion(raw_count, designmatrix, 'MAQC')

In [4]:
MAQC_disp.main()

In [5]:
import plotly.graph_objects as go
fig = go.Figure()
Mu = (MAQC_disp.normalized_countNZ.mean(axis=1)).to_numpy()
sort_arg = np.argsort(Mu)
# MLE = ((MAQC_disp.gwdispNZ).to_numpy())[sort_arg]
# raw_countC = MAQC_disp.raw_count
# raw_countC['sum'] = raw_countC.sum(axis=1)
# final = MAQC_disp.disp_res[raw_countC['sum'] != 0].to_numpy().squeeze()[sort_arg]
trend = np.exp(MAQC_disp.logalpha).to_numpy()[sort_arg]




In [6]:
MLE = pd.read_csv('Deseq_Disp_Est_Genewise.csv')['deseq2']
MLE[pd.isna(MLE)] = 0
MLE = MLE[MLE!=0]
MLE = ((MLE).to_numpy())[sort_arg]

final = pd.read_csv('MAQC_dispersion.csv')['0']
final[pd.isna(final)] = 0
final = final[final!=0]
final = ((final).to_numpy())[sort_arg]

In [7]:
fig.add_trace(go.Scatter(x=Mu, y=final, mode='markers', name='final_est',marker=dict(
            color='LightSkyBlue',
            size=2,
            opacity=0.5,
            line=dict(
                color='Red',
                width=2
            )
        )))
fig.add_trace(go.Scatter(x=Mu, y=MLE, mode='markers', name='gene_est',marker=dict(
            color='LightSkyBlue',
            size=2,
            opacity=0.5,
            line=dict(
                color='MediumPurple',
                width=2
            )
        )))
fig.add_trace(go.Scatter(x=Mu[sort_arg], y=trend, mode='lines', name='disp_trend',marker=dict(
            color='LightSkyBlue',
            size=1,
            opacity=0.5
            
        )))
fig.update_layout(
    autosize=False,
    width=1000,
    height=500)
fig.update_xaxes(title_text='mean of normalized counts',type='log')
fig.update_yaxes(title_text='dispersion',type='log')
fig.show()